# Testing for mbi

This scripts tests all of the python functions and dependencies in mbi. 

In [6]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
import imp
%matplotlib inline

import training
import merge
import predict_single_pass
import analyze_performance
import build_ensemble
import impute_markers
import models
import utils
import datetime

# Allows for fast correction
imp.reload(training)
imp.reload(merge)
imp.reload(predict_single_pass)
imp.reload(analyze_performance)
imp.reload(build_ensemble)
imp.reload(impute_markers)
imp.reload(models)
imp.reload(utils)

/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/analyze_performance.py:6: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start(

<module 'utils' from '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/utils.py'>

## Dataset and run folder pathing 

In [2]:
data_path = '/n/holylfs02/LABS/olveczky_lab/Diego/data/JDM25/20170916/test/dataset.h5'
base_output_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models'

## Train models

* Train n_models to be used in an ensemble. 
* Keep epochs short to save time during testing. 

In [3]:
n_models = 2;
for i in range(n_models):
    training.train(data_path, base_output_path=base_output_path, epochs=1, stride=5)

Loading Data
(1625364,)
(1625364, 60)
(1625364, 20)
(1312887, 60)
(1312887, 20)
(262578, 60)
(262578, 20)
Getting indices
Compiling network
data_name: dataset
run_name: dataset-wave_net_epochs=1_input_9_output_1
Building run folders
Created folder: /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/dataset-wave_net_epochs=1_input_9_output_1_03
Saving training info
Saving initial network
Training
Train on 147057 samples, validate on 25951 samples
Epoch 1/1
147057/147057 [==============================] - 89s 604us/step - loss: 0.0975 - mean_squared_error: 0.0975 - val_loss: 0.0663 - val_mean_squared_error: 0.0663

Epoch 00001: val_loss improved from inf to 0.06626, saving model to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/dataset-wave_net_epochs=1_input_9_output_1_03/best_model.h5
Total runtime: 1.5 mins
Saving final model
Loading Data
(1625364,)
(1625364, 60)
(1625364, 20)
(1312887, 60)
(1312887, 20)
(262578, 60)
(26257

## Build model ensemble

In [4]:
models_in_ensemble = ["dataset-wave_net_epochs=1_input_9_output_1_03/best_model.h5",
                     "dataset-wave_net_epochs=1_input_9_output_1_04/best_model.h5"]
build_ensemble.build_ensemble(base_output_path,*models_in_ensemble)

run_name: model_ensemble
Building run folders
Created folder: /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02
Saving training info
Saving model ensemble


In [ ]:
# Test to see if the model has been created. 
from keras.models import load_model
model = load_model('/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/final_model.h5')

## Impute markers

In [7]:
model_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/final_model.h5'
save_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions'
preds = impute_markers.impute_markers(model_path, data_path,save_path = save_path, n_frames=5000)

Loading data
Predicting 5000 frames starting at frame 0.
Loading model


/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Imputing markers: forward pass
Predicting frame:  0
Imputing markers: reverse pass
Predicting frame:  0
Computing weighted average
Saving to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions


## Impute Markers with chunked passes

In [8]:
model_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/final_model.h5'
save_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions'
n_folds = 1000;
n_fold_ids = 5;
for i in range(n_fold_ids):
    preds = predict_single_pass.predict_single_pass(model_path,data_path,'forward',save_path = save_path, n_folds = 1000, fold_id = i)
    preds = predict_single_pass.predict_single_pass(model_path,data_path,'reverse',save_path = save_path, n_folds = 1000, fold_id = i)

Loading data
Loading model
Predicting 1625 frames starting at frame 0.
Imputing markers: forward pass
Predicting frame:  0


/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '
/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1434: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


Saving to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions/forward_fold_id_0.mat
Loading data
Loading model
Predicting 1625 frames starting at frame 0.
Imputing markers: reverse pass
Predicting frame:  0
Saving to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions/reverse_fold_id_0.mat
Loading data
Loading model
Predicting 1625 frames starting at frame 1625.
Imputing markers: forward pass
Predicting frame:  0
Saving to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions/forward_fold_id_1.mat
Loading data
Loading model
Predicting 1625 frames starting at frame 1625.
Imputing markers: reverse pass
Predicting frame:  0
Saving to /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_02/predictions/reverse_fold_id_1.mat
Loading data
Loading model
Predicting 1625 frames startin

## Merge chunks

In [73]:
save_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/merged.h5'
fold_paths = ['/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/forward_fold_id_0.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/forward_fold_id_1.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/forward_fold_id_2.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/forward_fold_id_3.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/forward_fold_id_4.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/reverse_fold_id_0.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/reverse_fold_id_1.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/reverse_fold_id_2.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/reverse_fold_id_3.mat',
              '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/reverse_fold_id_4.mat']
merged = merge.merge(save_path,*fold_paths)

0
1
2
3
4
5
6
7
8
9
(8100, 60)
(8100, 60)
(8100, 60)
(8100, 60)
(1, 60)
(1, 60)
Computing weighted average
marker number: 0
marker number: 1
marker number: 2
marker number: 3
marker number: 4
marker number: 5
marker number: 6
marker number: 7
marker number: 8
marker number: 9
marker number: 10
marker number: 11
marker number: 12
marker number: 13
marker number: 14
marker number: 15
marker number: 16
marker number: 17
marker number: 18
marker number: 19
marker number: 20
marker number: 21
marker number: 22
marker number: 23
marker number: 24
marker number: 25
marker number: 26
marker number: 27
marker number: 28
marker number: 29
marker number: 30
marker number: 31
marker number: 32
marker number: 33
marker number: 34
marker number: 35
marker number: 36
marker number: 37
marker number: 38
marker number: 39
marker number: 40
marker number: 41
marker number: 42
marker number: 43
marker number: 44
marker number: 45
marker number: 46
marker number: 47
marker number: 48
marker number: 49
mar

## Load saved results

In [8]:
results_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/chunks/merged.h5'
with h5py.File(results_path,'r') as f:
    preds = f['preds'][:]
    print('preds: ', preds.shape)
    markers = f['markers'][:]
    print('markers: ', markers.shape)
    badFrames = f['badFrames'][:]
    print('badFrames: ', badFrames.shape)
    member_stds = f['member_stds'][:]
    print('member_stds: ', member_stds.shape)

preds:  (8100, 60)
markers:  (8100, 60)
badFrames:  (8100, 20)
member_stds:  (8100, 60)


## Analyze performance

In [4]:
model_base_path = '/n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01'
analyze_performance.analyze_performance(model_base_path, data_path, model_name='final_model.h5', analyze_history=False, stride=5, skip=1500)

Created run: 18_11_26_17_00_17_454199
Created viz directory: /n/holylfs02/LABS/olveczky_lab/Diego/code/MarkerBasedImputation/mbi/test_models/model_ensemble_01/viz/18_11_26_17_00_17_454199
Loading model


/n/home02/daldarondo/.conda/envs/tf1.4_cuda8/lib/python3.6/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Loading data
Getting indices 0
(102445, 9)
Indexing into data
Predicting marker  0
Finished predictions of marker 0 in 208.747741 seconds
Predicting marker  1
Finished predictions of marker 1 in 0.254334 seconds
Predicting marker  2
Finished predictions of marker 2 in 0.258758 seconds
Predicting marker  3
Finished predictions of marker 3 in 0.257706 seconds
Predicting marker  4
Finished predictions of marker 4 in 0.252962 seconds
Predicting marker  5
Finished predictions of marker 5 in 0.251517 seconds
Predicting marker  6
Finished predictions of marker 6 in 0.260722 seconds
Predicting marker  7
Finished predictions of marker 7 in 0.254066 seconds
Predicting marker  8
Finished predictions of marker 8 in 0.252847 seconds
Predicting marker  9
Finished predictions of marker 9 in 0.252969 seconds
Predicting marker  10
Finished predictions of marker 10 in 0.261209 seconds
Predicting marker  11
Finished predictions of marker 11 in 0.252651 seconds
Predicting marker  12
Finished predictions o

Finished predictions of marker 5 in 0.546328 seconds
Predicting marker  6
Finished predictions of marker 6 in 0.544920 seconds
Predicting marker  7
Finished predictions of marker 7 in 0.546121 seconds
Predicting marker  8
Finished predictions of marker 8 in 0.534824 seconds
Predicting marker  9
Finished predictions of marker 9 in 0.546830 seconds
Predicting marker  10
Finished predictions of marker 10 in 0.545907 seconds
Predicting marker  11
Finished predictions of marker 11 in 0.544786 seconds
Predicting marker  12
Finished predictions of marker 12 in 0.538914 seconds
Predicting marker  13
Finished predictions of marker 13 in 0.546825 seconds
Predicting marker  14
Finished predictions of marker 14 in 0.545394 seconds
Predicting marker  15
Finished predictions of marker 15 in 0.545880 seconds
Predicting marker  16
Finished predictions of marker 16 in 0.532315 seconds
Predicting marker  17
Finished predictions of marker 17 in 0.545888 seconds
Predicting marker  18
Finished predictions 